### Visualizing the distribution of the observations

### Load the required libraries

In [1]:
import pandas as pd
import numpy as np
import sklearn as sk
import urllib
import math
%pylab inline

import findspark
findspark.init()

from pyspark import SparkContext
#sc.stop()
sc = SparkContext(master="local[3]",pyFiles=['lib/numpy_pack.py','lib/spark_PCA.py','lib/computeStats.py'])

from pyspark import SparkContext
from pyspark.sql import *
sqlContext = SQLContext(sc)

import sys
sys.path.append('./lib')

import numpy as np
from numpy_pack import packArray,unpackArray
from spark_PCA import computeCov
from computeStats import computeOverAllDist, STAT_Descriptions

### Read the data frame from pickle file

data_dir='../../Data/Weather'
file_index='SSSBSSSS'

from pickle import load

#read statistics
filename=data_dir+'/STAT_%s.pickle'%file_index
STAT,STAT_Descriptions = load(open(filename,'rb'))
#print 'keys from STAT=',STAT.keys()

#read data
filename=data_dir+'/US_Weather_%s.parquet'%file_index

df=sqlContext.read.parquet(filename)
print df.count()
df.show(5)

Populating the interactive namespace from numpy and matplotlib
12741
+---------+--------+---------+-----------+-----------+------+--------------------+------+--------+
|elevation|latitude|longitude|measurement|    station|undefs|              vector|  year|   label|
+---------+--------+---------+-----------+-----------+------+--------------------+------+--------+
|    402.9| 36.4708|-121.1472|       TMAX|USR0000CPNN|     2|[B8 59 00 58 38 5...|2002.0|SSSBSSSS|
|    402.9| 36.4708|-121.1472|       TMAX|USR0000CPNN|     2|[90 59 10 5A 20 5...|2003.0|SSSBSSSS|
|    402.9| 36.4708|-121.1472|       TMAX|USR0000CPNN|     1|[E0 55 00 58 E0 5...|2004.0|SSSBSSSS|
|    402.9| 36.4708|-121.1472|       TMAX|USR0000CPNN|     2|[50 57 90 55 A0 5...|2005.0|SSSBSSSS|
|    402.9| 36.4708|-121.1472|       TMAX|USR0000CPNN|     7|[A0 56 F0 56 A0 5...|2006.0|SSSBSSSS|
+---------+--------+---------+-----------+-----------+------+--------------------+------+--------+
only showing top 5 rows



### Select data for a particular station and measurement type

In [2]:
sqlContext.registerDataFrameAsTable(df,'weather')
Query1="SELECT * FROM weather\n\tWHERE measurement='%s'"%('SNWD')
print Query1
df_test = sqlContext.sql(Query1)
print df_test.count(),'rows'
df_test.show()
rows_test=df_test.rdd.map(lambda row:(row.elevation,row.station,unpackArray(row['vector'],np.float16))).collect()
#rows_test1=rows_test.map(lambda (v,y):(v,np.hstack(y))).collect()
 #xs: [x[0] for x in xs]

                         #T=np.vstack(rows_test
#T=T/10.  # scaling to make the temperature be in centingrates
#shape(T)


import pandas as pd

df_test=pd.DataFrame(rows_test)

df_test1=df_test[2].apply(pd.Series)/10

# rename 0 to elvation 

del df_test[2]

df_test3=pd.concat([df_test,df_test1],axis=1)

df_test3

SELECT * FROM weather
	WHERE measurement='SNWD'
2111 rows
+---------+--------+---------+-----------+-----------+------+--------------------+------+--------+
|elevation|latitude|longitude|measurement|    station|undefs|              vector|  year|   label|
+---------+--------+---------+-----------+-----------+------+--------------------+------+--------+
|     29.0| 36.9308|-121.7692|       SNWD|USC00049473|     0|[00 00 00 00 00 0...|1949.0|SSSBSSSS|
|     29.0| 36.9308|-121.7692|       SNWD|USC00049473|     0|[00 00 00 00 00 0...|1950.0|SSSBSSSS|
|     29.0| 36.9308|-121.7692|       SNWD|USC00049473|    11|[00 00 00 00 00 0...|1951.0|SSSBSSSS|
|     29.0| 36.9308|-121.7692|       SNWD|USC00049473|     3|[00 00 00 00 00 0...|1952.0|SSSBSSSS|
|     29.0| 36.9308|-121.7692|       SNWD|USC00049473|     4|[00 00 00 00 00 0...|1953.0|SSSBSSSS|
|     29.0| 36.9308|-121.7692|       SNWD|USC00049473|     0|[00 00 00 00 00 0...|1954.0|SSSBSSSS|
|     29.0| 36.9308|-121.7692|       SNWD|USC000494

,0,1,0,1,2,3,4,5,6,7,...,355,356,357,358,359,360,361,362,363,364
0,29.0,USC00049473,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,29.0,USC00049473,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,29.0,USC00049473,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,29.0,USC00049473,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,29.0,USC00049473,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,29.0,USC00049473,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,29.0,USC00049473,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,29.0,USC00049473,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,29.0,USC00049473,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,29.0,USC00049473,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
df_test4=df_test3.iloc[:,2:]
result=df_test4.mean(axis=1)
type(result)

pandas.core.series.Series

In [7]:
df_test6=df_test3.iloc[:,:2]

In [8]:
df_test5=result.to_frame(name='Avg SNWD')
df_test5.shape

(2111, 1)

In [9]:
reg_data=pd.concat([df_test6, df_test5], axis=1)

In [10]:
reg_data.columns = ['Elevation', 'Station', 'Avg SNWD']
reg_data

,Elevation,Station,Avg SNWD
0,29.0,USC00049473,0.000000
1,29.0,USC00049473,0.000000
2,29.0,USC00049473,0.000000
3,29.0,USC00049473,0.000000
4,29.0,USC00049473,0.000000
5,29.0,USC00049473,0.000000
6,29.0,USC00049473,0.000000
7,29.0,USC00049473,0.000000
8,29.0,USC00049473,0.000000
9,29.0,USC00049473,0.000000


In [8]:
import statsmodels.api as sm

## Regression

In [11]:
from pandas.stats.api import ols
regression = ols(y=reg_data['Avg SNWD'], x=reg_data['Elevation'])
regression

/Users/mengtingwang/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2881: FutureWarning: The pandas.stats.ols module is deprecated and will be removed in a future version. We refer to external packages like statsmodels, see some examples here: http://www.statsmodels.org/stable/regression.html
  exec(code_obj, self.user_global_ns, self.user_ns)



-------------------------Summary of Regression Analysis-------------------------

Formula: Y ~ <x> + <intercept>

Number of Observations:         2111
Number of Degrees of Freedom:   2

R-squared:         0.2272
Adj R-squared:     0.2268

Rmse:              0.0546

F-stat (1, 2109):   619.9341, p-value:     0.0000

Degrees of Freedom: model 1, resid 2109

-----------------------Summary of Estimated Coefficients------------------------
      Variable       Coef    Std Err     t-stat    p-value    CI 2.5%   CI 97.5%
--------------------------------------------------------------------------------
             x     0.0001     0.0000      24.90     0.0000     0.0001     0.0001
     intercept    -0.0138     0.0015      -9.36     0.0000    -0.0167    -0.0109
---------------------------------End of Summary---------------------------------

In [19]:
#sc.stop()